In [56]:
import pymongo

from tqdm import tqdm

In [12]:
client = pymongo.MongoClient()
db = client.soccerdb

In [13]:
db.events.find_one()

{'_id': ObjectId('5d9dced91c801fea8e22e72e'),
 'eventId': 8,
 'eventName': 'Pass',
 'eventSec': 1.255989999999997,
 'id': 88178642,
 'matchId': 1694390,
 'matchPeriod': '1H',
 'playerId': 26010,
 'positions': [{'x': 50, 'y': 48}, {'x': 47, 'y': 50}],
 'subEventId': 85,
 'subEventName': 'Simple pass',
 'tags': [{'id': 1801}],
 'teamId': 4418}

In [17]:
def count_some_event(event, matchId=None, teamId=None, playerId=None):
    cond = {'eventName': event}
    if matchId is not None:
        cond.update({'matchId': matchId})
    if teamId is not None:
        cond.update({'teamId': teamId})
    if playerId is not None:
        cond.update('playerId')
        
    return db.events.find(cond, {'_id': 0}).count()

### Get a set of all (matchId, teamId) tuples
We need the set of all matches and the corresponding teams to work with.

In [42]:
set_matches2teams = set(map(lambda dic: (dic['matchId'], dic['teamId']), db.events.find(
    {'eventName': 'Pass', 'matchPeriod': '1H'}, 
    {'_id': 0, 'matchId': 1, 'teamId': 1})))

### Get the list of all events
Find 3 different matchIds to find the set of events inside them. This helps decreasing the probability of not seeing an event due to it's absence in a random game.

In [52]:
some_matchids = [x['matchId'] for x in db.events.find({}, {'_id': 0, 'matchId': 1}, limit=3)]
set_event_names = set([x['eventName'] for x in db.events.find({'matchId': 1694430}, {'_id': 0, 'eventName': 1})])

In [57]:
for matchId, teamId in tqdm(set_matches2teams):
    base = counts = {
        'matchId': matchId,
        'teamId': teamId
    }
    for event in set_event_names:
        counts.update({
            'num{}'.format(event.title().replace(' ', '')): count_some_event(event, matchId, teamId)
        })
    db.results.update(counts, base, upsert=True)

  0%|          | 0/102 [00:00<?, ?it/s]/home/mohammad/projects/soccer-prediction/venv/lib/python3.5/site-packages/ipykernel_launcher.py:10: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  # Remove the CWD from sys.path while we load stuff.
/home/mohammad/projects/soccer-prediction/venv/lib/python3.5/site-packages/ipykernel_launcher.py:10: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  # Remove the CWD from sys.path while we load stuff.
100%|██████████| 102/102 [00:56<00:00,  1.81it/s]
